In [ ]:
import os
import pydicom
import numpy as np
import torch 
import glob
import sys
import time 

import matplotlib.pyplot as plt
import matplotlib.path as mplPath

import pylab as pl

from threading import Timer

from collections import Counter
from collections import defaultdict

## Inspect a folder

In [ ]:
class Folder(object): 
    
    def __init__(self, root, subject_substr=""): 
        
        if "\\" in root: 
            raise Exception("Please replace the backslashes in your path with forwardslashes.")
        self.root = root
        self.subfolders = self.get_subjects(subject_substr)
        
        self.walk_history = []
        self.walk_history.append(self.subfolders)

        self.walk_history_lost = []
        self.walk_history_lost.append([])
        
     
    def get_folders_files(self, path):
        for root, folders, files in os.walk(path):
            break   
        
        return folders, files
        
    def get_subjects(self, subject_substr):         
        subjects, _ = self.get_folders_files(self.root)
        
        return [s for s in subjects if subject_substr in s]

    def down(self, equals=[], contains=[], does_not_contain=[], print_absent=False): 
        if not isinstance(equals, list) or not isinstance(contains, list) or not isinstance(does_not_contain, list):
            raise Exception("Please enter a list with strings instead of a string.")
        
        print(f"\nSubfolder equals {str(equals)},   contains {str(contains)},   does not contain {str(does_not_contain)}")
        subfolders_present = []
        subfolders_absent = []
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            
            folders, files = self.get_folders_files(path)
            
            for string in equals: 
                folders = [f for f in folders if string.lower() == f.lower()]
            
            for string in contains: 
                folders = [f for f in folders if string.lower() in f.lower()]
            
            for string in does_not_contain: 
                folders = [f for f in folders if string.lower() not in f.lower()]
            
            if len(folders) == 0: 
                subfolders_absent.append(subfolder)
                
            elif len(folders) == 1: 
                subfolders_present.append(os.path.join(subfolder, folders[0]))
                
            elif len(folders) > 1: 
                print(f"\n{subfolder} has multiple subfolders with the specified name:")
                print(folders)
                subfolders_present.append(os.path.join(subfolder, folders[0]))
        
        if len(subfolders_present) == 0:
            print("\nNo folders contained the specified subfolders, therefore this step is not executed.")
        
        else:    
            print(f"\n{len(subfolders_present)} of {len(self.subfolders)} folders contained specified subfolder.")
            if print_absent: 
                print("\nFolders without subfolder")
                for s in subfolders_absent:
                    print("\t", s)

            self.walk_history.append(subfolders_present)
            self.walk_history_lost.append(subfolders_absent)
            self.subfolders = subfolders_present
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
        
            
    def up(self): 
        if len(self.walk_history) == 1:
            print("You can not go up as you are already at the subject folder")
        
        else: 
            self.subfolders = self.walk_history[-2]
            self.walk_history = self.walk_history[:-1]
            self.walk_history_lost = self.walk_history_lost[:-1]  
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
    
    def sample_down(self, i=0): 
        subfolder = self.subfolders[i]
        path = os.path.join(self.root, subfolder)
            
        folders, files = self.get_folders_files(path)
        print("\nCurrent path depth:", subfolder)
        print("Subfolders:")
        for s in folders: 
            print("\t", s)
    
    def current_path(self):
        return self.subfolders[0]
            
    def number_of_files(self, print_dict=False):
        
        dct = defaultdict(list)
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            _, files = self.get_folders_files(path)
            length = len(files)
            
            dct[str(length)].append(subfolder)
        
        for l, f in dct.items():
            print(f"\n{str(len(f))} folders contain {l} files each.")
            
        if print_dict: 
            for key, value in dct.items(): 
                print(f"Folders with {key} files:")
                for s in value: 
                    print("\t", s)

## Save images as arrays

In [ ]:
def save_3d_images(folder_class, save_to):
    for string in sorted(folder_class.subfolders):
        # get subject name and path
        subject, loc = string.split("/", 1)
        path = os.path.join(folder_class.root, string)
        
        # get all files in folder
        for root, folders, files in os.walk(path):
            break  
        files = sorted(files)
        
        # create 3d image
        three_d = []
        for image in files:
            if "DICOMDIR" not in image:
                ds = pydicom.dcmread(os.path.join(path, image))
                pix = ds.pixel_array
                three_d.append(pix)

        if len(three_d) != 0: 
            three_d = np.dstack(three_d)

            # create subfolder if not existent
            if not os.path.exists(save_to):
                os.makedirs(save_to)

            # save np array
            with open(f'{save_to}/{subject}.npy', 'wb') as f:
                np.save(f, three_d)